In [50]:
#have the dataframes all at once 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

def process_file(file):
    col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]
    
    for i in range(1, 33):
        col_names.append(f"Sp{i}")
    
    df = pd.read_csv(file, names=col_names, sep='\s+', header=None)
    df = df.set_index('Index')
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
    df = df[df["MonStatus"] == 1]

    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6}
    df['DateM'] = df['DateM'].str[:3].map(month_map)
    df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
    df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')

    df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
                                     month=df['Date'].dt.month,
                                     day=df['Date'].dt.day,
                                     hour=df['Time'].dt.hour,
                                     minute=df['Time'].dt.minute,
                                     second=df['Time'].dt.second))

    df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)

    day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}

    df.insert(0, 'Day', df['Time'].dt.day.map(day_map))
    
    return df

def process_files(files):
    dataframes = {}
    for idx, file in enumerate(files, start=1):
        df = process_file(file)
        dataframes[f'df{idx}'] = df
    return dataframes

files = ['Steatoda B masking 12 pm.txt']

dataframes = process_files(files)
for name, df in dataframes.items():
    print(f"{name}:")
    display(df)


df1:


,Day,Time,Light,Sp1,Sp2,Sp3,Sp4,Sp5,Sp6,Sp7,...,Sp23,Sp24,Sp25,Sp26,Sp27,Sp28,Sp29,Sp30,Sp31,Sp32
Index,,,,,,,,,,,,,,,,,,,,,
2,1,2024-03-13 19:09:00,1,0,1,0,0,3,0,0,...,3,0,0,0,0,0,0,0,0,0
3,1,2024-03-13 19:10:00,1,0,1,0,3,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,2024-03-13 19:11:00,1,0,1,0,2,3,0,1,...,0,0,0,0,0,0,0,0,0,0
5,1,2024-03-13 19:12:00,1,0,1,0,4,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,2024-03-13 19:13:00,1,3,0,0,1,1,0,4,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9585,8,2024-03-20 10:52:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9586,8,2024-03-20 10:53:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9587,8,2024-03-20 10:54:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
df_masking = dataframes['df1']
light_column = df_masking['Light']

difference = np.diff(light_column)
df_masking['Light2'] = np.append(difference, np.nan)

lights_on = df_masking.loc[df_masking.Light2 == -1]
display(lights_on)

lights_off = df_masking.loc[df_masking.Light2 == 1]
display(lights_off)

,Day,Time,Light,Sp1,Sp2,Sp3,Sp4,Sp5,Sp6,Sp7,...,Sp24,Sp25,Sp26,Sp27,Sp28,Sp29,Sp30,Sp31,Sp32,Light2
Index,,,,,,,,,,,,,,,,,,,,,
51,1,2024-03-13 19:58:00,1,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,-1.0
1491,2,2024-03-14 19:58:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0
2931,3,2024-03-15 19:58:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0
4371,4,2024-03-16 19:58:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0
5334,5,2024-03-17 12:01:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0
5811,5,2024-03-17 19:58:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0
7251,6,2024-03-18 19:58:00,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0
8691,7,2024-03-19 19:58:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0


,Day,Time,Light,Sp1,Sp2,Sp3,Sp4,Sp5,Sp6,Sp7,...,Sp24,Sp25,Sp26,Sp27,Sp28,Sp29,Sp30,Sp31,Sp32,Light2
Index,,,,,,,,,,,,,,,,,,,,,
771,2,2024-03-14 07:58:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
2210,3,2024-03-15 07:57:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
3650,4,2024-03-16 07:57:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
5090,5,2024-03-17 07:57:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
5461,5,2024-03-17 14:08:00,0,0,0,0,0,12,0,0,...,0,0,0,0,0,0,0,0,0,1.0
6530,6,2024-03-18 07:57:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
7970,7,2024-03-19 07:57:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
9410,8,2024-03-20 07:57:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
